In [ ]:
import pandas as pd
import requests
import openpyxl


from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.events import EventFiringWebDriver, AbstractEventListener
from selenium.webdriver.common.action_chains import ActionChains

# 1. 진짜 크롤링 및 저장 함수

In [ ]:
# 데이터 담을 변수 선언 (전역변수??? 뭐시긴가)
data1 = []
data2 = []
data3 = []
data4 = []


# 수집함수
def crawler(url, location):     
    driver = webdriver.Chrome('./chromedriver')
    driver.implicitly_wait(1)
    driver.get(url)


    # 밑작업1 : 왼쪽 매물정보 한 번 클릭(수집 시 스크롤 내리는 명령 위한 밑밥)
    ele = driver.find_element_by_xpath('//*[@id="listContents1"]/div/div/div[1]/div[1]/div')
    time.sleep(0.5)
    ele.click()


    # '상가' 단어에 접근해 텍스트 가져오기
    try:
        for i in range(1, 200): # 본격수집 : 00동 한 구역에 200개가 안되서 반복 돌리고, 다 수집하면 오류메시지 받아서 수집 종료시키는 로직
            t1 = driver.find_element_by_css_selector('#listContents1 > div > div > div > div:nth-child(' + str(i) + ') > div > a > div.item_title > span').text
            t2 = driver.find_element_by_css_selector('#listContents1 > div > div > div > div:nth-child(' + str(i) + ') > div > a > div.price_line > span.price').text
            t3 = driver.find_element_by_css_selector('#listContents1 > div > div > div > div:nth-child(' + str(i) + ') > div > a > div.info_area > p:nth-child(1) > span').text
            t4 = driver.find_element_by_css_selector('#listContents1 > div > div > div > div:nth-child(' + str(i) + ') > div > div.cp_area > div > span:nth-child(2)').text
            print(t1, t2, t3, t4) # 수집내역 확인을 위한 Print문

            
            # 수집 내용 변수에 저장 (DataFrame 관점에서 보면 '->' 가로 방향으로 데이터 쌓게되는 비효율?)
            data1.append(t1)
            data2.append(t2)
            data3.append(t3)
            data4.append(t4)
        
        
            # 밑작업2 : 스크롤 얼마나 내려야할 지 명확하지 않아 반복시마다 END키로 좌측 매물공간 스크롤 내림 (비효율)
            ActionChains(driver).key_down(Keys.COMMAND).send_keys(Keys.END).perform()

            
    except : # 오류나거나 반복문 돌기 전 수집이 끝나면 메시지 띄움
        print('{} 수집종료!!!!!!!!!'.format(location))

        
    finally: # 오류잡아서 해당 00동 데이터 수집 종료
        pass
    

# 데이터 Export함수
def saveFile(location):
    result = [data1, data2, data3, data4] # 수집된 데이터 받아서 list 안에 list형태
    dataframe = pd.DataFrame(result)
    dataframe = dataframe.T # (세로로 쌓여야하는 데이터가 가로로 쌓여서 Transpose시켜 정상적인 DF형태로 변환)
    dataframe.columns = ['Type', 'Size', 'Height', 'Provider'] # 칼럼이름 지정
    
    
    filename = location + ".xlsx"
    dataframe.to_excel(filename, sheet_name = 'Sheet1', index = False, encoding = 'utf-8 sig')
    
    
    # 변수가 함수 바깥에 있어서, 데이터가 누적쌓이지 않게 지워줌 (result변수는 함수 내 선언되어서 안해도 됨)
    del data1[:]
    del data2[:]
    del data3[:]
    del data4[:]

# 2. 수집대상 '00구 00동'에 대한 url 및 지역이름

In [ ]:
url = ['https://new.land.naver.com/offices?ms=37.5115,126.9416,16&a=SG:SMS:GJCG:APTHGJ:GM:TJ&e=RETAIL&f=100000',
'https://new.land.naver.com/offices?ms=37.508133,126.92635,16&a=SG:SMS:GJCG:APTHGJ:GM:TJ&e=RETAIL&f=100000',
'https://new.land.naver.com/offices?ms=37.494973,126.976016,16&a=SG:SMS:GJCG:APTHGJ:GM:TJ&e=RETAIL&f=100000',
'https://new.land.naver.com/offices?ms=37.513058,126.956088,16&a=SG:SMS:GJCG:APTHGJ:GM:TJ&e=RETAIL&f=100000',
'https://new.land.naver.com/offices?ms=37.4853,126.9719,16&a=SG:SMS:GJCG:APTHGJ:GM:TJ&e=RETAIL&f=100000',
'https://new.land.naver.com/offices?ms=37.506842,126.951,16&a=SG:SMS:GJCG:APTHGJ:GM:TJ&e=RETAIL&f=100000',
'https://new.land.naver.com/offices?ms=37.4991,126.9437,16&a=SG:SMS:GJCG:APTHGJ:GM:TJ&e=RETAIL&f=100000',
'https://new.land.naver.com/offices?ms=37.4921,126.9172,16&a=SG:SMS:GJCG:APTHGJ:GM:TJ&e=RETAIL&f=100000',
'https://new.land.naver.com/offices?ms=37.5052,126.9629,16&a=SG:SMS:GJCG:APTHGJ:GM:TJ&e=RETAIL&f=100000',
'https://new.land.naver.com/offices?ms=37.5366,126.8759,16&a=SG:SMS:GJCG:APTHGJ:GM:TJ&e=RETAIL&f=100000',
'https://new.land.naver.com/offices?ms=37.527925,126.8342,16&a=SG:SMS:GJCG:APTHGJ:GM:TJ&e=RETAIL&f=100000',
'https://new.land.naver.com/offices?ms=37.516433,126.8632,16&a=SG:SMS:GJCG:APTHGJ:GM:TJ&e=RETAIL&f=100000',
'https://new.land.naver.com/offices?ms=37.474539,126.977837,16&a=SG:SMS:GJCG:APTHGJ:GM:TJ&e=RETAIL&f=100000',
'https://new.land.naver.com/offices?ms=37.4753,126.9567,16&a=SG:SMS:GJCG:APTHGJ:GM:TJ&e=RETAIL&f=100000',
'https://new.land.naver.com/offices?ms=37.4632,126.9346,16&a=SG:SMS:GJCG:APTHGJ:GM:TJ&e=RETAIL&f=100000',
'https://new.land.naver.com/offices?ms=37.483351,126.887738,16&a=SG:SMS:GJCG:APTHGJ:GM:TJ&e=RETAIL&f=100000',
'https://new.land.naver.com/offices?ms=37.494393,126.856813,16&a=SG:SMS:GJCG:APTHGJ:GM:TJ&e=RETAIL&f=100000',
'https://new.land.naver.com/offices?ms=37.5024,126.8598,16&a=SG:SMS:GJCG:APTHGJ:GM:TJ&e=RETAIL&f=100000',
'https://new.land.naver.com/offices?ms=37.4937,126.8823,16&a=SG:SMS:GJCG:APTHGJ:GM:TJ&e=RETAIL&f=100000',
'https://new.land.naver.com/offices?ms=37.5009,126.8277,16&a=SG:SMS:GJCG:APTHGJ:GM:TJ&e=RETAIL&f=100000',
'https://new.land.naver.com/offices?ms=37.507766,126.880583,16&a=SG:SMS:GJCG:APTHGJ:GM:TJ&e=RETAIL&f=100000',
'https://new.land.naver.com/offices?ms=37.492143,126.838375,16&a=SG:SMS:GJCG:APTHGJ:GM:TJ&e=RETAIL&f=100000',
'https://new.land.naver.com/offices?ms=37.4938,126.8207,16&a=SG:SMS:GJCG:APTHGJ:GM:TJ&e=RETAIL&f=100000',
'https://new.land.naver.com/offices?ms=37.4802,126.8391,16&a=SG:SMS:GJCG:APTHGJ:GM:TJ&e=RETAIL&f=100000',
'https://new.land.naver.com/offices?ms=37.481,126.8258,16&a=SG:SMS:GJCG:APTHGJ:GM:TJ&e=RETAIL&f=100000']


location = ['동작구 노량진동', '동작구 대방동', '동작구 동작동', '동작구 본동', '동작구 사당동', '동작구 상도1동', '동작구 상도동', '동작구 신대방동', '동작구 흑석동','양천구 목동', '양천구 신월동', '양천구 신전동','관악구 남현동', '관악구 신림동', '관악구 신림동','구로구 가리봉동', '구로구 개봉동', '구로구 고척동', '구로구 구로동', '구로구 궁동', '구로구 신도림동', '구로구 오류동', '구로구 온수동', '구로구 천왕동', '구로구 향동']


print(len(url))
print(len(location))

# 3. 25개 지역, 데이터 수집 자동화 명령문

In [ ]:
for url, location in zip(url, location):
    crawler(url, location)
    saveFile(location)

### 방법2 : Excel 라이브러리 사용해 저장하는 로직 (사용 안함)

In [ ]:
excel_file = openpyxl.Workbook()
excel_sheet = excel_file.active

url = 'https://new.land.naver.com/offices?ms=37.4993979,126.9407067,17&a=SG:SMS:GJCG:APTHGJ:GM:TJ&e=RETAIL&f=100000'
driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(1)
driver.get(url)


# 왼쪽 매물정보 클릭 후 스크롤 내리기
ele = driver.find_element_by_xpath('//*[@id="listContents1"]/div/div/div[1]/div[1]/div')
ele.click()
ele.click()


for i in range(1, 5):
    ActionChains(driver).key_down(Keys.COMMAND).send_keys(Keys.END).perform()

    
# '상가' 단어에 접근해 텍스트 가져오기
try:
    for i in range(1, 10):

        data1 = {"Type" : []}
        data2 = {"Size" : []}
        data3 = {"Location" : []}
        data4 = {"Provider" : []}


        t1 = driver.find_element_by_css_selector('#listContents1 > div > div > div > div:nth-child(' + str(i) + ') > div > a > div.item_title > span').text
        t2 = driver.find_element_by_css_selector('#listContents1 > div > div > div > div:nth-child(' + str(i) + ') > div > a > div.price_line > span.price').text
        t3 = driver.find_element_by_css_selector('#listContents1 > div > div > div > div:nth-child(' + str(i) + ') > div > a > div.info_area > p:nth-child(1) > span').text
        t4 = driver.find_element_by_css_selector('#listContents1 > div > div > div > div:nth-child(' + str(i) + ') > div > div.cp_area > div > span:nth-child(2)').text
        print(t1, t2, t3, t4)
        
        
        excel_sheet.append([t1, t2, t3, t4])
        
        
except:
    print('뉴스 수집 종료!')

    
finally:
    pass


excel_file.save('NoTank.xlsx')
excel_file.close()
    
time.sleep(1)